In [31]:
%matplotlib inline
import os
import pandas as pd
import glob
import numpy as np

In [40]:
import librosa
import librosa.display
import pylab
import matplotlib
import gc
import matplotlib.pyplot as plt

In [41]:
class Opts():
    def __init__(self):
        self.train_data_path = "./working/train/"
        self.test_data_path = "./working/test/"
        self.csv_path = "./meta"
        self.data_path = "./data/"
        self.model_save_path = "./output/"
        self.model_name = "cnn.h5"
        self.sample_rate = 22050
        self.hop_length = 512
        self.window_size = 1024
        
        if not os.path.exists(self.train_data_path):
            os.makedirs(self.train_data_path)
        if not os.path.exists(self.test_data_path):
            os.makedirs(self.test_data_path)
        if not os.path.exists(self.csv_path):
            os.makedirs(self.csv_path)


In [42]:
CLASS_ID = {0: "air_conditioner",
            1: "car_horn",
            2:"children_playing",
            3:"dog_bark",
            4:"drilling",
            5:"engine_idling",
            6:"gun_shot",
            7:"jackhammer",
            8:"siren",
            9:"street_music"}

def _get_meta_info(filename):
    infos = filename.split('-')
    return [int(info) for info in infos]

In [43]:
def input_to_target(opts):
    # audio files and their corresponding labels
    train_paths = [opts.data_path + "fold1/*.wav", opts.data_path + "fold2/*.wav"]
    # train_paths = [opts.data_path + "fold1/*.wav"]
    train_label_path = opts.data_path +  "train_labels.csv"
    test_paths =  [opts.data_path + "fold3/*.wav"]

    # input
    train_files, test_files = [], []
    for train_path in train_paths:
        train_files += glob.glob(train_path)
    for test_path in test_paths:
        test_files += glob.glob(test_path)

    train_labels, class_names, file_names = [], [], []
    for train_file in train_files:
        _, class_id, _, _ = _get_meta_info(train_file.split('/')[-1].strip('.wav'))
        # print("train_file={:s}, class_id={:d}".format(train_file, class_id))
        train_labels.append(int(class_id))
        class_names.append(CLASS_ID[int(class_id)])
        file_names.append(train_file.split('/')[-1].split('.')[0])
    # csv storing information for training dataset
    train_file_df = pd.DataFrame({'file_paths': train_files,
                                  'file_names': file_names,
                                  'labels': train_labels,
                                  'class_names': class_names})

    test_labels, class_names, file_names = [], [], []
    for test_file in test_files:
        _, class_id, _, _ = _get_meta_info(test_file.split('/')[-1].strip('.wav'))
        test_labels.append(int(class_id))
        class_names.append(CLASS_ID[int(class_id)])
        file_names.append(test_file.split('/')[-1].split('.')[0])
    # csv storing information for training dataset
    test_file_df = pd.DataFrame({'file_paths': test_files, 
                                 'file_names': file_names,
                                 'labels': test_labels,
                                 'class_names': class_names})
    
    train_file_df.to_csv(os.path.join(opts.csv_path, "train.csv"))
    test_file_df.to_csv(os.path.join(opts.csv_path, "test.csv"))

    return train_file_df, test_file_df

In [44]:
def create_spectrogram(file_path, title, opts, flag="train"):
    plt.interactive(False)
    clip, _ = librosa.load(file_path, sr=opts.sample_rate)
    clip_normed = librosa.util.normalize(clip)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111) #subplot 1: 1x1 grid

    # hide all axes
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)

    S = librosa.feature.melspectrogram(y=clip_normed, sr=opts.sample_rate,\
                                       hop_length=self.hop_length, window_size=self.window_size)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))

    if flag == "train":
        filename  = os.path.join(opts.train_data_path, title + '.jpg')
    else:
        filename  = os.path.join(opts.test_data_path, title + '.jpg')

    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')

In [45]:
def gen_spectrogram_set(df, opts, flag="train"):
    for file_path in df['file_paths']:
        title = file_path.split('/')[-1].split('.')[0]
        create_spectrogram(file_path, title, opts, flag)

In [46]:
opts = Opts()

In [47]:

train_file_df, test_file_df = input_to_target(opts)
gen_spectrogram_set(train_file_df, opts)
gen_spectrogram_set(test_file_df, opts, flag="test")

In [78]:
def df_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    return df
train_csv_path = os.path.join(opts.csv_path, "train.csv")
test_csv_path = os.path.join(opts.csv_path, "test.csv")

train_file_df, test_file_df = df_from_csv(train_csv_path), df_from_csv(test_csv_path)

In [79]:
from keras_preprocessing.image import ImageDataGenerator
data_gen = ImageDataGenerator(rescale=1./255., validation_split=0.15) #normalize data by multiplying 1/255

def append_ext(fn):
    return fn+".jpg"
train_file_df["file_names"] = train_file_df["file_names"].apply(append_ext)
test_file_df["file_names"] = test_file_df["file_names"].apply(append_ext)

train_gen = data_gen.flow_from_dataframe(
    dataframe=train_file_df,
    directory=opts.train_data_path,
    x_col="file_names",
    y_col="class_names",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64, 64))

valid_gen = data_gen.flow_from_dataframe(
    dataframe=train_file_df,
    directory=opts.train_data_path,
    x_col="file_names",
    y_col="class_names",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64, 64))

Found 1497 images belonging to 10 classes.
Found 264 images belonging to 10 classes.


In [75]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from tensorflow.python.keras.utils.data_utils import Sequence

In [51]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_24 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_25 (Activation)   (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 31, 31, 64)        36928     
__________

In [52]:
#Fitting keras model, no test gen for now
STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)
model.evaluate_generator(generator=valid_gen, steps=STEP_SIZE_VALID
)

Epoch 1/20
46/46 [==============================] - 44s 963ms/step - loss: 2.2299 - acc: 0.1670 - val_loss: 2.0503 - val_acc: 0.2266

[0.65720234811306, 0.8203125]

In [102]:
test_data_gen = ImageDataGenerator(rescale=1./255.)
test_gen=test_data_gen.flow_from_dataframe(
    dataframe=test_file_df,
    directory=opts.test_data_path,
    x_col="file_names",
    y_col="class_names",
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=(64,64))
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size
test_gen.reset() # resets batch index to 0

Found 925 images belonging to 10 classes.


In [103]:
test_gen.classes

[9,
 1,
 5,
 5,
 3,
 6,
 2,
 5,
 9,
 3,
 7,
 4,
 1,
 4,
 9,
 7,
 7,
 9,
 6,
 4,
 5,
 5,
 0,
 8,
 8,
 8,
 0,
 3,
 4,
 7,
 7,
 7,
 2,
 0,
 8,
 8,
 8,
 0,
 5,
 4,
 5,
 7,
 7,
 3,
 4,
 0,
 4,
 0,
 5,
 4,
 9,
 4,
 5,
 5,
 3,
 1,
 5,
 2,
 3,
 5,
 5,
 3,
 4,
 3,
 4,
 5,
 4,
 5,
 4,
 0,
 2,
 7,
 8,
 8,
 8,
 7,
 0,
 3,
 5,
 7,
 7,
 0,
 8,
 8,
 6,
 8,
 8,
 7,
 0,
 0,
 4,
 5,
 5,
 4,
 4,
 0,
 4,
 0,
 6,
 5,
 9,
 4,
 2,
 4,
 2,
 1,
 2,
 5,
 6,
 7,
 5,
 3,
 9,
 9,
 4,
 2,
 4,
 5,
 4,
 5,
 2,
 7,
 4,
 2,
 4,
 5,
 3,
 7,
 8,
 3,
 4,
 0,
 1,
 5,
 5,
 8,
 8,
 0,
 3,
 8,
 8,
 3,
 0,
 8,
 8,
 2,
 5,
 0,
 6,
 3,
 8,
 9,
 6,
 4,
 3,
 5,
 4,
 0,
 0,
 6,
 4,
 8,
 2,
 4,
 5,
 4,
 9,
 3,
 7,
 5,
 1,
 4,
 2,
 4,
 1,
 9,
 5,
 7,
 9,
 6,
 0,
 9,
 5,
 5,
 4,
 4,
 9,
 4,
 3,
 4,
 9,
 5,
 3,
 4,
 0,
 0,
 5,
 0,
 3,
 8,
 0,
 2,
 6,
 8,
 3,
 7,
 7,
 1,
 7,
 5,
 7,
 4,
 8,
 7,
 0,
 8,
 5,
 3,
 4,
 3,
 5,
 3,
 2,
 4,
 3,
 0,
 4,
 2,
 5,
 5,
 2,
 3,
 4,
 2,
 9,
 7,
 5,
 6,
 2,
 8,
 2,
 7,
 9,
 8,
 9,
 7,
 5,
 8,
 6,
 7,


In [105]:
pred=model.predict_generator(test_gen,
    steps=STEP_SIZE_TEST,
    verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
predicted_class_indices
#Fetch labels from train gen for testing
# labels = (train_gen.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in predicted_class_indices]
# print(predictions[0:6])


28/28 [==============================] - 9s 318ms/step

array([2, 1, 5, 7, 3, 6, 3, 5, 9, 3, 4, 4, 1, 4, 9, 9, 7, 2, 6, 4, 5, 5,
       8, 8, 8, 8, 5, 3, 4, 7, 4, 7, 3, 5, 8, 8, 8, 8, 5, 4, 5, 7, 0, 3,
       4, 5, 4, 5, 5, 7, 9, 2, 7, 5, 6, 1, 2, 2, 3, 5, 5, 3, 4, 3, 4, 5,
       4, 5, 4, 8, 2, 5, 8, 8, 8, 7, 0, 3, 2, 7, 7, 5, 8, 8, 6, 8, 8, 5,
       5, 8, 4, 5, 5, 4, 4, 5, 4, 5, 6, 7, 8, 9, 2, 4, 3, 1, 3, 8, 6, 4,
       3, 8, 3, 3, 1, 3, 4, 9, 9, 5, 3, 2, 4, 2, 4, 5, 3, 4, 8, 2, 8, 8,
       4, 5, 2, 8, 8, 0, 3, 8, 8, 3, 0, 8, 8, 6, 5, 8, 9, 3, 8, 9, 9, 4,
       3, 5, 4, 5, 5, 6, 4, 8, 3, 5, 7, 1, 9, 3, 8, 9, 1, 7, 3, 4, 1, 9,
       3, 7, 2, 6, 7, 2, 7, 7, 4, 4, 9, 4, 3, 4, 2, 5, 8, 4, 8, 8, 5, 8,
       3, 8, 0, 2, 2, 8, 2, 0, 1, 1, 7, 9, 7, 4, 8, 4, 0, 8, 5, 3, 4, 5,
       5, 3, 2, 4, 3, 5, 4, 3, 7, 7, 3, 3, 4, 2, 2, 9, 2, 6, 6, 8, 2, 5,
       7, 8, 3, 2, 5, 8, 3, 9, 4, 5, 2, 2, 8, 8, 2, 5, 7, 5, 4, 3, 3, 7,
       9, 8, 1, 2, 7, 2, 2, 7, 8, 1, 8, 8, 4, 7, 7, 5, 3, 6, 4, 8, 7, 1,
       9, 2, 8, 9, 4, 5, 4, 8, 3, 5, 7, 8, 9, 6, 5,

In [86]:
test_gen

In [84]:
train_gen

In [85]:
if not os.path.exists(opts.model_save_path):
    os.mkdir(opts.model_save_path)
model.save(os.path.join(opts.model_save_path, opts.model_name))

In [59]:
model = load_model(os.path.join(opts.model_save_path, opts.model_name))

In [54]:
import coremltools
coreml_model = coremltools.converters.keras.convert(model,
                                                    input_names="image", 
                                                    output_names="probability",
                                                    class_labels="labels.txt", 
                                                    image_input_names="image")
coreml_model.author = "Candy Dong"
coreml_model.short_description = "model for sound recognition"
coreml_model.save(os.path.join(opts.model_save_path, "CNN.mlmodel"))


0 : conv2d_19_input, <keras.engine.input_layer.InputLayer object at 0x1c39e1ea20>
1 : conv2d_19, <keras.layers.convolutional.Conv2D object at 0x1c39e04cf8>
2 : activation_24, <keras.layers.core.Activation object at 0x1c39e04550>
3 : conv2d_20, <keras.layers.convolutional.Conv2D object at 0x1c39e1eb00>
4 : activation_25, <keras.layers.core.Activation object at 0x1c39e1e9e8>
5 : max_pooling2d_10, <keras.layers.pooling.MaxPooling2D object at 0x1c3f95da20>
6 : conv2d_21, <keras.layers.convolutional.Conv2D object at 0x1c3f95d748>
7 : activation_26, <keras.layers.core.Activation object at 0x1c39e1eac8>
8 : conv2d_22, <keras.layers.convolutional.Conv2D object at 0x1c3b2b1da0>
9 : activation_27, <keras.layers.core.Activation object at 0x1c3b2b1ac8>
10 : max_pooling2d_11, <keras.layers.pooling.MaxPooling2D object at 0x1c3ac9d0b8>
11 : conv2d_23, <keras.layers.convolutional.Conv2D object at 0x1c3ac9d978>
12 : activation_28, <keras.layers.core.Activation object at 0x1c3b2b1b38>
13 : conv2d_24, <k

In [63]:
# post analysis
# retrieve weights from the first conv layer
filters, biases = model.layers[0].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plot first few filters
n_filters, ix = 10, 1
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(3):
        # specify subplot and turn of axis
        ax = pyplot.subplot(n_filters, 1, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel
        pyplot.imshow(f[:, :, j])
        ix += 1
# show the figure
pyplot.show()

NameError: name 'pyplot' is not defined

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_24 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_25 (Activation)   (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 31, 31, 64)        36928     
__________